<a href="https://colab.research.google.com/github/lxa215/Covid_19_Shiny_App/blob/master/Sample_Matching_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Imports

import pandas as pd
import os
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [25]:
# Retrieve the CONFIDENTIAL patient dataset with test results

# You must mount your drive if you have access to the ***CONFIDENTIAL PATIENT TEST RESULTS***
test_result_array = []
test_results_path = "./drive/My Drive/patient-test-results/pcr-result"

for dirpath, dnames, fnames in os.walk(test_results_path):
    for f in fnames:
        if f.endswith(".csv"):
            print(f)
            file_path = os.path.join(test_results_path, f)
            test_result_array += [pd.read_csv(file_path)] # Converts to a dataframe
   




COVID-19 14-04-2020.xlsx - Sheet1.csv
COVID-19 13-04-2020 (sanitized).csv


In [9]:
# Read the Patient Raw Data 

# Retrieve the JSON
!wget --user 'lambda' --password 'W$RCc&cmQ^n!b4k' 'https://detect-now.org/pakistan/url/org/admin_access_data'



--2020-04-15 18:06:55--  https://detect-now.org/pakistan/url/org/admin_access_data
Resolving detect-now.org (detect-now.org)... 52.178.92.96
Connecting to detect-now.org (detect-now.org)|52.178.92.96|:443... connected.
HTTP request sent, awaiting response... 401 UNAUTHORIZED
Authentication selected: Basic realm="Login Required"
Reusing existing connection to detect-now.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 33649 (33K) [application/json]
Saving to: ‘admin_access_data’

admin_access_data   100%[===================>]  32.86K   145KB/s    in 0.2s    

2020-04-15 18:06:57 (145 KB/s) - ‘admin_access_data’ saved [33649/33649]



In [26]:
# Convert to a df

json_df = pd.read_json(r'/content/admin_access_data')
json_df.head()


,age,consent,corona_test,cough_filename,gender,medical_history,patientID,patient_id,patient_reported_symptoms,sampleType,data_t,smoker,video_patient_finder_tip_filename,breathe_filename,patient_smartphone_used_for_data_collection
0,41 - 55,yes,Negative,https://detectnowdatalake.blob.core.windows.ne...,male,"Diabetes with complications,",DMC-202,637,"Fever, chills, or sweating,New or worsening co...",study,NaN,NaN,NaN,NaN,NaN
1,31 - 40,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,female,Disease or conditions that make it harder to c...,20030511135111357902,648,"New or worsening cough,Sore throat,",study,NaN,NaN,NaN,NaN,NaN
2,31 - 40,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,male,,200413111464227,649,"Fever, chills, or sweating,",study,NaN,NaN,NaN,NaN,NaN
3,18 - 30,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,male,,200413111464200,650,,guest,NaN,NaN,NaN,NaN,NaN
4,31 - 40,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,male,,200413111463989,652,"New or worsening cough,Loss of taste,Loss of s...",study,NaN,NaN,NaN,NaN,NaN


In [27]:
#TO DO: iterate through all the files provided by the hospital for matching


daily_df_hospital = test_result_array[0]
daily_df_hospital.head()

,Unnamed: 0,Unnamed: 1,COVID DATA 14/04/2020,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,S.NO,LAB No,MR No,INVOCE,NAME,AGE,GENDER,CoV-2 PCR Result,DATE,Contact No,NaN
1,2056,8016252911,200413-111465599,111625291,AZHAR HUSSAIN,29,MALE,NOT DETECTED,13/04/2020,3000732059,NaN
2,2057,8016252561,200413-111465593,111625256,DR MUKHTIAR AHMED,40,MALE,NOT DETECTED,13/04/2020,3331373971,NaN
3,2058,8016253911,200413-111465629,111625391,DR SYED MAZHAR HUSSSAIN,36,MALE,NOT DETECTED,13/04/2020,3333022032,NaN
4,2059,8016252561,200413-111465593,111625256,DR. AMBREEN,40,FEMALE,NOT DETECTED,13/04/2020,3331373971,NaN


In [31]:
# Code to count the number of matches!

#TO DO: iterate through all the files provided by the hospital for matching

counter = 0
match_counter = 0

for patient_id in json_df['patientID']:
  counter_1 = 0
  for patient_id_2 in daily_df['COVID DATA 14/04/2020']: #Change the date column 
    patient_id = str(patient_id).replace('-' , '')
    patient_id_2 = str(patient_id_2).replace('-' , '')
    if patient_id == patient_id_2:
      json_df['corona_test'][counter] = daily_df_hospital['Unnamed: 7'][counter_1] # Change the result column 
      match_counter += 1
    counter_1 += 1
  counter += 1
 




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [30]:
# Anonymize Output

deleted_patient_id = json_df.drop(columns='patient_id')
deleted_PatientID = deleted_patient_id.drop(columns='patientID')

deleted_PatientID.head()

,age,consent,corona_test,cough_filename,gender,medical_history,patient_reported_symptoms,sampleType,data_t,smoker,video_patient_finder_tip_filename,breathe_filename,patient_smartphone_used_for_data_collection
0,41 - 55,yes,Negative,https://detectnowdatalake.blob.core.windows.ne...,male,"Diabetes with complications,","Fever, chills, or sweating,New or worsening co...",study,NaN,NaN,NaN,NaN,NaN
1,31 - 40,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,female,Disease or conditions that make it harder to c...,"New or worsening cough,Sore throat,",study,NaN,NaN,NaN,NaN,NaN
2,31 - 40,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,male,,"Fever, chills, or sweating,",study,NaN,NaN,NaN,NaN,NaN
3,18 - 30,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,male,,,guest,NaN,NaN,NaN,NaN,NaN
4,31 - 40,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,male,,"New or worsening cough,Loss of taste,Loss of s...",study,NaN,NaN,NaN,NaN,NaN


In [33]:
# Count the Percentage of Correctly Matched
#counter = 0
#for result in deleted_PatientID['corona_test']:
# if result[-1].isupper():
#    counter += 1



print('The percentage of correctly mapped raw files is: ', round(match_counter/len(raw_df['patient_id']) * 100. ,2) , '%') 

The percentage of correctly mapped raw files is:  44.44 %


In [0]:
# Store as a .csv
filename = 'output-04-15-2020-1.csv'
deleted_PatientID.to_csv('/content/drive/My Drive/patient-test-results/curated-outputs/' + filename )